In [ ]:
# import autosklearn.classification
from pipeline import *

# pipeline_year(10)

df_coaches = pd.read_csv('dataset/cleaned/coaches.csv')
l = []
clf = RandomForestClassifier(n_estimators=100, random_state=42)
decay_rate=0.1
df_teams_merged = []
test = []
train = []
year = 10
for i in range(2, year + 1):
    df_teams_merged = pipeline_clf(year=i)
    weight = decay_rate ** (10 - i - 1)

    df_teams_merged['confID'] = df_teams_merged['confID'].replace({'EA': 0, 'WE': 1})

    train = df_teams_merged[df_teams_merged['year'] < i]
    test = df_teams_merged[df_teams_merged['year'] == i]

    if (year == i):
        break

    X_train = train[train.drop(['playoff', 'year', 'tmID'], axis=1).columns]
    y_train = train['playoff']
    sample_weight = [weight] * len(X_train)

    clf.fit(X_train, y_train, sample_weight=sample_weight)

predictions = clf.predict_proba(test.drop(['playoff', 'year', 'tmID'], axis=1))[:, 1]
test['predictions'] = predictions
df_teams_merged['predictions'] = 0
df_teams_merged.loc[df_teams_merged['year'] == year, 'predictions'] = predictions

df_teams_merged['confID'] = df_teams_merged['confID'].replace({0: 'EA', 1 : 'WE'})

# print the year and the predicted scores
print(df_teams_merged[df_teams_merged['year'] == year][['tmID', 'confID', 'predictions', 'awards', 'offensive_strength']].sort_values(by='predictions', ascending=False))


df_teams, ea_teams, we_teams = classify_playoff_entry(
        df_teams_merged, year)

ea_predictions = ea_teams['tmID'].unique()
we_predictions = we_teams['tmID'].unique()


accuracy = calculate_playoff_accuracy(
    year, ea_predictions, we_predictions, display_results = True)

print(l)


In [ ]:
pipeline_year(8,display_results=True)

In [ ]:
from pipeline import *

def check_accuracy_by_year2():
    accs = []
    years = list(range(2, 11))

    for year in years:
        acc = pipeline_year(year)
        accs.append(acc)

    # plot the accuracy line graph
    plt.plot(years, accs, label="Accuracy", marker='o', linestyle='-')

    # add labels for each data point
    # for i, acc in enumerate(accs):
      #  plt.text(years[i], acc, f"{acc:.2f}", ha="center", va="bottom")

    # add legend
    plt.legend()

    # set Y-axis limits
    plt.ylim(0, 100)

    plt.xlabel("Year")
    plt.ylabel("Accuracy")
    plt.title("Accuracy by year")
    plt.grid(True)

    plt.show()

check_accuracy_by_year2()